Links:
1. https://minimaxir.com/2017/06/reddit-deep-learning/ 
2. https://www.freecodecamp.org/news/how-to-predict-likes-and-shares-based-on-your-articles-title-using-machine-learning-47f98f0612ea/
3. https://towardsdatascience.com/another-twitter-sentiment-analysis-with-python-part-4-count-vectorizer-b3f4944e51b5

Notes: need more feature engineering (scrape or add data), also need to clean data for special characters, etc...

Problems: dealing with text data, model doesnt work at all

In [1]:
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer as tfidf
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

Using TensorFlow backend.


In [3]:
tech = pd.read_csv(r"C:\Users\Jenna\Documents\medium-tech-data.csv")

In [4]:
tech = tech.drop(tech.columns[[0, 10, 11]], axis = 1) #drop null columns
tech = tech.drop(['url', 'username'], axis = 1) #drop unused columns (for now)
tech = tech[~(tech.text == '[]')]
tech = tech.dropna() #drop null rows

### Features

For simplicity purposes for now,
I will just look at the text column. Will add more later

(probably need to scrape/add more features such as follower data, etc.)


In [5]:
tech2 = tech[['claps', 'text']]
tech2.head()

,claps,text
0,155,The agricultural food production mechanisms an...
1,83,After many years of overdone bass and lacklust...
2,197,"Welcome to Bad Ideas, a column in which we exa..."
3,197,"Welcome to Bad Ideas, a column in which we exa..."
4,827,"Welcome to Bad Ideas, a column in which we exa..."


In [6]:
# cleaning - making all claps into integer value

def convert_to_number(x):
    total_stars = 0
    if 'K' in x:
        if len(x) > 1:
            total_stars = float(x.replace('K', '')) * 1000 # convert k to a thousand
    else:
        try:
            total_stars = int(x) # Less than 1000
        except ValueError:
            return None
    return int(total_stars)

tech2['claps'] = tech2['claps'].apply(convert_to_number)
tech2 = tech2.dropna()

C:\Users\Jenna\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [7]:
#splitting into training, validation, test (60/20/20)
x = tech2.text
y = tech2.claps
SEED = 2000

#split train and val/test
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.4, random_state=SEED)
#split val and test
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

In [8]:
#vectorize text
tvec = tfidf(max_features = 100000)
x_train_tfidf = tvec.fit_transform(x_train)
x_validation_tfidf = tvec.transform(x_validation)

In [9]:
#check: do we need to normalize data?

In [10]:
#model building - needs work
def batch_generator_shuffle(X_data, y_data, batch_size):
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    index = np.arange(np.shape(y_data)[0])
    np.random.shuffle(index)
    while 1:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X_data[index_batch,:].toarray()
        y_batch = y_data[y_data.index[index_batch]]
        counter += 1
        yield X_batch,y_batch
        if (counter > number_of_batches):
            np.random.shuffle(index)
            counter=0
            
model_s = Sequential()
model_s.add(Dense(64, activation='relu', input_dim=20676))
model_s.add(Dense(64, activation='relu'))
model_s.add(Dense(1))

optimizer = tf.keras.optimizers.RMSprop(0.001)
model_s.compile(optimizer=optimizer,
              loss='mse',
              metrics=['accuracy'])

model_s.fit_generator(generator=batch_generator_shuffle(x_train_tfidf, y_train, 32),
                    epochs=5, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

Epoch 1/5
75/74 [==============================] - 2s 27ms/step - loss: 7765097.9024 - accuracy: 0.0054 - val_loss: 3010785.3504 - val_accuracy: 0.0063
Epoch 2/5
75/74 [==============================] - 2s 22ms/step - loss: 7740360.6374 - accuracy: 0.0033 - val_loss: 2985714.7882 - val_accuracy: 0.0025
Epoch 3/5
75/74 [==============================] - 1s 20ms/step - loss: 7710827.4692 - accuracy: 0.0025 - val_loss: 2942573.0236 - val_accuracy: 0.0025
Epoch 4/5
75/74 [==============================] - 1s 20ms/step - loss: 7636372.5715 - accuracy: 0.0021 - val_loss: 2883547.7967 - val_accuracy: 0.0013
Epoch 5/5
75/74 [==============================] - 2s 20ms/step - loss: 7562863.7999 - accuracy: 8.3438e-04 - val_loss: 2820065.8671 - val_accuracy: 0.0000e+00


In [14]:
example_batch = x_train_tfidf[:10]
example_result = model_s.predict(example_batch)
example_result

array([[205.0973 ],
       [156.43466],
       [201.00728],
       [193.64778],
       [166.66046],
       [156.98335],
       [314.93533],
       [202.71512],
       [239.74243],
       [196.07767]], dtype=float32)